# Should I eat There?

## Data Layer

#### Loading Data

Using Food Inspection Data from the [City of Austin Open Data Portal](https://data.austintexas.gov/dataset/Restaurant-Inspection-Scores/ecmv-9xxi)

In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None

In [ ]:
import os.path

restaurants = pd.read_csv('./Restaurant_Inspection_Scores.csv')

restaurants['Inspection Date'] = pd.to_datetime(restaurants['Inspection Date'])

def to_letter_score(row):
    if row['Score'] >= 90:
        return 'A'
    elif row['Score'] >= 80:
        return 'B'
    elif row['Score'] >= 70:
        return 'C'
    elif row['Score'] >= 60:
        return 'D'
    else:
        return 'F'
    
restaurants['Letter Score'] = restaurants.apply(to_letter_score, axis=1)

In [ ]:
restaurants.head()

In [ ]:
print("Total restaurants: {}".format(restaurants['Restaurant Name'].count()))

In [ ]:
print("Minimum score: {}".format(restaurants['Score'].min()))

In [ ]:
restaurants[restaurants['Score']==36]

In [ ]:
print("Averate score: {}".format(restaurants['Score'].mean()))

#### Extracting Geo Location
The `Address` column contains the `lat`/`long` of the restaurant

In [ ]:
lat_long_pattern = '\((.*),(.*)\)'

In [ ]:
restaurants_with_lat_long = restaurants[restaurants['Address'].str.contains(lat_long_pattern)]

In [ ]:
restaurants_with_lat_long.head()

In [ ]:
print("Total restaurants with lat/long: {}".format(restaurants_with_lat_long['Restaurant Name'].count()))

In [ ]:
restaurants_with_lat_long.loc[:, 'Geo Location'] = restaurants_with_lat_long['Address'].str.split('\n').str[-1]

In [ ]:
lat_long = restaurants_with_lat_long['Geo Location'].str.extract(lat_long_pattern)
restaurants_with_lat_long.loc[:, 'Latitude'] = lat_long[0]
restaurants_with_lat_long.loc[:, 'Longitude'] = lat_long[1]

In [ ]:
restaurants_with_lat_long.head()

In [ ]:
restaurants_with_lat_long_and_last_score = restaurants_with_lat_long.sort_values(['Inspection Date','Facility ID']).drop_duplicates(subset='Facility ID', keep='last')
restaurants_with_lat_long_and_last_score.head()

#### Find Restaurants close to a location

In [ ]:
target = (30.317743, -97.721324)

In [ ]:
from math import sin, cos, sqrt, atan2, radians

In [ ]:
# approximate radius of earth in km
R = 6373.0
    
# returns the distance between 2 lat/log coordinates. Units based on R
def distance( start, end ):
    start_rad = tuple(map(radians, start))
    end_rad = tuple(map(radians, end))
    

    dlon = end_rad[1] - start_rad[1]
    dlat = end_rad[0] - start_rad[0]

    a = sin(dlat / 2)**2 + cos(start_rad[0]) * cos(start_rad[0]) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c

In [ ]:
distance(target, (30.432288, -97.675307))

In [ ]:
from functools import partial

columns_for_near = ['Facility ID', 
                    'Restaurant Name', 
                    'Zip Code', 
                    'Address', 
                    'Score', 
                    'Inspection Date',
                    'Latitude', 
                    'Longitude',
                    'Letter Score']

def distance_to( restaurant, target):
    rest_loc = (float(restaurant['Latitude']), float(restaurant['Longitude']))
    return distance(target, rest_loc)

def get_near(lat:float, long:float, radius=0.5):
    df = restaurants_with_lat_long_and_last_score
    distance_to_partial = partial(distance_to, target=(lat,long))
    distances = df.apply(distance_to_partial, axis=1)
    return df.loc[distances[distances < radius].sort_values().index.values][columns_for_near]

In [ ]:
get_near(target[0],target[1])

#### Getting specific Restaurants history of inspection scores

In [ ]:
a_facility_id = 2800676

In [ ]:
a_restaurant = restaurants[restaurants['Facility ID'] == a_facility_id]
a_restaurant

In [ ]:
a_score = a_restaurant[["Inspection Date", "Score"]]

In [ ]:
a_sorted_score = a_score.sort_values('Inspection Date', inplace=False)
a_sorted_score

In [ ]:
def get_scores(id):
    a_restaurant = restaurants[restaurants['Facility ID'] == int(id)]
    a_score = a_restaurant.sort_values('Inspection Date', inplace=False)
    return a_score[["Inspection Date", "Score"]]

In [ ]:
get_scores(a_facility_id)

#### Getting specific Restaurants based on facility id

Returning a simple dictionary and fixing column names to avoid spaces

In [ ]:
def rest_from_id(id):
    df = restaurants_with_lat_long_and_last_score
    dict_recs = df[df['Facility ID'] == int(id)].to_dict(orient='records')
    
    a_rest = dict_recs[0] if dict_recs else None
    the_a_rest = {}
    
    if a_rest:
        for key in a_rest.keys():
            the_a_rest[key.replace(' ','_').lower()] = a_rest[key]
    
    return the_a_rest

In [ ]:
rest_from_id(a_facility_id)

#### Search for restaurants by name

In [ ]:
restaurants_with_lat_long_and_last_score[restaurants_with_lat_long_and_last_score['Restaurant Name'].str.lower().str.contains("vas")]

In [ ]:
def get_by_name(search: "", lat:float, long:float, radius=10):
    df = get_near(lat,long, radius)
    search_hits_restaurants = df[df['Restaurant Name'].str.lower().str.contains(search.lower())][columns_for_near]
    return search_hits_restaurants

In [ ]:
get_by_name("Vas", target[0],target[1], 10.0)

## Interactive Layer

#### Dependencies and global setup

In [ ]:
from declarativewidgets import init
init()

In [ ]:
%%html

<link rel="import" href="urth_components/google-map-observe/google-map.html" is='urth-core-import' package='lbustelo/google-map-observe#ObserveNodesMarkers'>
<link rel="import" href="urth_components/google-map-observe/google-map-marker.html" is='urth-core-import' package='lbustelo/google-map-observe#ObserveNodesMarkers'>
<link rel="import" href="urth_components/paper-slider/paper-slider.html" is='urth-core-import' package='PolymerElements/paper-slider'>
<link rel="import" href="urth_components/paper-progress/paper-progress.html" is='urth-core-import' package='PolymerElements/paper-progress'>
<link rel="import" href="urth_components/paper-item/paper-item.html" is='urth-core-import' package='PolymerElements/paper-item'>
<link rel="import" href="urth_components/paper-item/paper-item-body.html" is='urth-core-import' package='PolymerElements/paper-item-body'>
<link rel="import" href="urth_components/paper-scroll-header-panel/paper-scroll-header-panel.html" is='urth-core-import' package='PolymerElements/paper-scroll-header-panel'>
<link rel="import" href="urth_components/paper-toolbar/paper-toolbar.html" is='urth-core-import' package='PolymerElements/paper-toolbar'>
<link rel="import" href="urth_components/paper-icon-button/paper-icon-button.html" is='urth-core-import' package='PolymerElements/paper-icon-button'>
<link rel="import" href="urth_components/paper-input/paper-input.html" is='urth-core-import' package='PolymerElements/paper-input'>
<link rel="import" href="urth_components/iron-selector/iron-selector.html" is='urth-core-import' package='PolymerElements/iron-selector'>
<link rel="import" href="urth_components/iron-icon/iron-icon.html" is='urth-core-import' package='PolymerElements/iron-icon'>
<link rel="import" href="urth_components/urth-viz-line/urth-viz-line.html" is='urth-core-import'>

<style is="custom-style">
    google-map,
    paper-scroll-header-panel {
        height: 500px;
    }
    
    paper-progress {
        display: inline-block;
        width: 85px;
        margin-bottom: 3px;
    }
    
    .rest-details paper-progress {
        width: 125px;
    }
    
    paper-toolbar paper-input {
        --paper-input-container-input-color: white;
    }
</style>

Association of necessary images from bundling
```
images/*.png
```

In [ ]:
from declarativewidgets import channel

channel().watch("sel_rest_id", lambda old, new: channel().set("sel_rest", rest_from_id(new)))

#### Map to find restaurants close by

In [ ]:
%%html
<template id="mapTemplate" is="urth-core-bind">
    <urth-core-function
        id="get_near"
        ref="get_near" 
        arg-lat="[[center_lat]]" 
        arg-long="[[center_long]]" 
        arg-radius="1.5" 
        result="{{near}}" limit="50" delay="2000"></urth-core-function>
    <google-map id="theMap" latitude="{{center_lat}}" longitude="{{center_long}}" 
        zoom="15" disable-zoom disable-default-ui single-info-window
        drag-events on-google-map-dragend="handleMapDragged"
        on-google-map-ready="handleMapReady">

        <template is="dom-repeat" items="[[near.data]]">
            <google-map-marker latitude="[[item.6]]" longitude="[[item.7]]" 
                title="[[item.1]]" data-restid$="[[item.0]]"
                icon="images/[[item.8]]_score.png"
                click-events on-google-map-marker-click="handleMarkerClick"></google-map-marker>
        </template>

        <template is="dom-if" if="[[sel_rest]]">
            <google-map-marker id="selMarker" latitude="[[sel_rest.latitude]]" longitude="[[sel_rest.longitude]]" 
                title="[[sel_rest.restaurant_name]]" data-restid$="[[item.facility_id]]"
                on-google-map-marker-open="handleMarkerOpen">
                    <div class="rest-details">
                        <paper-item data-restid$="[[sel_rest.facility_id]]">
                            <paper-item-body three-line>
                                <div>[[sel_rest.restaurant_name]]</div>
                                <div secondary>Score: [[sel_rest.score]] <paper-progress class="blue" value="[[sel_rest.score]]" style="display: inline-block;margin-bottom: 3px;"></paper-progress></div>
                                <div secondary>Last inspection: [[sel_rest.inspection_date]]</div>
                                <div secondary>[[sel_rest.address]]</div>
                            </paper-item-body>
                        </paper-item>
                        <urth-viz-line id="viz" width="500" height="300" datarows='{{scores.data}}' columns='{{scores.columns}}' ybounds="[60,100]">
                            <urth-viz-col index="0" type="date" format="%b %d %Y"></urth-viz-col>
                        </urth-viz-line>
                    </div>
            </google-map-marker>
        </template>
    </google-map>
    <script>
        mapTemplate.handleMarkerClick = function(e){
            var clickedMarker = e.target;
            this.channelElement.set('sel_rest_id', clickedMarker ? clickedMarker.dataset.restid : undefined);
        }
        
        mapTemplate.handleMapDragged = function(){
            if(!this.channelElement.get("name_search")){
                get_near.invoke() 
            }
        }

        mapTemplate.handleMapReady = function(){
            if(!this.channelElement.get("name_search")){
                this.async(function(){
                    get_near.invoke() 
                }, 1000) 
            }
        }
        
        mapTemplate.channelElement.watch("scores", function(){
            theMap.$$("#viz").datarows = mapTemplate.scores.data;
            if(selMarker.info){
                selMarker.info.open(selMarker.map, selMarker.marker);
            }
        })
        
        mapTemplate.handleMarkerOpen = function(){
           theMap.$$("#viz").datarows = mapTemplate.scores.data;     
        }
    </script>
</template>

In [ ]:
%%html
<template id="listTemplate" is="urth-core-bind">
    <urth-core-function
        id="get_by_name"
        ref="get_by_name" 
        arg-search="[[name_search]]"
        arg-lat="[[center_lat]]" 
        arg-long="[[center_long]]" 
        arg-radius="10"
        result="{{near}}" limit="50" delay="2000"></urth-core-function>
    <paper-scroll-header-panel>
        <paper-toolbar class="small-tall">
            <paper-input value="{{name_search}}">
              <iron-icon icon="search" prefix></iron-icon>
              <paper-icon-button suffix on-click="handleClearClick" icon="clear" alt="clear" title="clear"></paper-icon-button>
            </paper-input>
        </paper-toolbar>
        <div class="content" role="listbox">
            <iron-selector attr-for-selected="data-restid" selected="{{sel_rest_id}}" on-iron-select="handleSelected">
                <template is="dom-repeat" items="[[near.data]]">
                    <paper-item data-restid$="[[item.0]]" on-click="handleItemClick">
                        <paper-item-body three-line>
                            <div>[[item.1]]</div>
                            <div secondary>Score: [[item.4]] <paper-progress value="[[item.4]]"></paper-progress></div>
                            <div secondary>[[item.3]]</div>
                        </paper-item-body>
                    </paper-item>
                </template>
            </iron-selector>
        </div>
    </paper-scroll-header-panel>
    <script>
        listTemplate.handleSelected = function(e){
            e.detail.item.scrollIntoViewIfNeeded()
        }
        
        listTemplate.handleItemClick = function(e){
            if(!selMarker.open){
                selMarker.async(function(){
                    theMap.map.panTo(selMarker.marker.getPosition());
                }, 500);
            }
        }
        
        listTemplate.handleClearClick = function(){
            this.name_search = "";
            get_near.invoke();
        }
        
        listTemplate.channelElement.watch("name_search", function(){
            if(!!listTemplate.channelElement.get("name_search"))
                get_by_name.invoke();
        })
    </script>
</template>

#### Plot of restaurant scores

In [ ]:
%%html
<template id="scoreTemplate" is="urth-core-bind">
    <urth-core-function 
        ref="get_scores" 
        arg-id="[[sel_rest_id]]" 
        result="{{scores}}" auto></urth-core-function>
    <template is="dom-if" if="[[sel_rest]]">
        <div class="rest-details">
            <paper-item data-restid$="[[sel_rest.facility_id]]">
                <paper-item-body three-line>
                    <div>[[sel_rest.restaurant_name]]</div>
                    <div secondary>Score: [[sel_rest.score]] <paper-progress value="[[sel_rest.score]]"></paper-progress></div>
                    <div secondary>Last inspection: [[sel_rest.inspection_date]]</div>
                    <div secondary>[[sel_rest.address]]</div>
                </paper-item-body>
            </paper-item>
            <urth-viz-line width="500" height="300" datarows='{{scores.data}}' columns='{{scores.columns}}' ybounds="[60,100]">
                <urth-viz-col index="0" type="date" format="%b %d %Y"></urth-viz-col>
            </urth-viz-line>
        </div>
    </template>
</template>

#### Toolbar and Title area

In [ ]:
%%html
<paper-toolbar>
    <div class="title">Where should I eat?</div>
    <paper-icon-button icon="menu"></paper-icon-button>
</paper-toolbar>

#### Default values and debug

In [ ]:
%%html
<urth-core-channel debug>
    <urth-core-channel-item key="center_lat" value="30.4016717"></urth-core-channel-item>
    <urth-core-channel-item key="center_long" value="-97.7166573"></urth-core-channel-item>
</urth-core-channel>

## Layout Layer

Using `jupyter-dashboard` extension, layout the interactive cells into the following mockup.

![mockup](images/mockup.png)